In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torchvision
import torchvision.models as models
from PIL import Image
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [3]:
labels = ["apple_pie", "baby_back_ribs", "baklava", "beef_carpaccio", "beef_tartare", "beet_salad", "beignets", "bibimbap", "bread_pudding", "breakfast_burrito", "bruschetta", "caesar_salad", "cannoli", "caprese_salad", "carrot_cake", "ceviche", "cheese_plate", "cheesecake", "chicken_curry", "chicken_quesadilla", "chicken_wings", "chocolate_cake", "chocolate_mousse", "churros", "clam_chowder", "club_sandwich", "crab_cakes", "creme_brulee", "croque_madame", "cup_cakes", "deviled_eggs", "donuts", "dumplings", "edamame", "eggs_benedict", "escargots", "falafel", "filet_mignon", "fish_and_chips", "foie_gras", "french_fries", "french_onion_soup", "french_toast", "fried_calamari", "fried_rice", "frozen_yogurt", "garlic_bread", "gnocchi", "greek_salad", "grilled_cheese_sandwich", "grilled_salmon", "guacamole", "gyoza", "hamburger", "hot_and_sour_soup", "hot_dog", "huevos_rancheros", "hummus", "ice_cream", "lasagna", "lobster_bisque", "lobster_roll_sandwich", "macaroni_and_cheese", "macarons", "miso_soup", "mussels", "nachos", "omelette", "onion_rings", "oysters", "pad_thai", "paella", "pancakes", "panna_cotta", "peking_duck", "pho", "pizza", "pork_chop", "poutine", "prime_rib", "pulled_pork_sandwich", "ramen", "ravioli", "red_velvet_cake", "risotto", "samosa", "sashimi", "scallops", "seaweed_salad", "shrimp_and_grits", "spaghetti_bolognese", "spaghetti_carbonara", "spring_rolls", "steak", "strawberry_shortcake", "sushi", "tacos", "takoyaki", "tiramisu", "tuna_tartare", "waffles"]

In [7]:
test_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((224, 224)),
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

image = Image.open("../images/sushi.jpg")
image = image.convert('RGB')
image = test_transforms(image)
image = torch.reshape(image, (1, 3, 224, 224))
image = image.to(device)

model_name = "unfreeze_{}.pth".format(56)
checkpoint = torch.load("../models/{}".format(model_name), map_location="cpu")

model = models.resnet34(pretrained=False)

model.fc = nn.Linear(in_features=512, out_features=101, bias=True)
model.load_state_dict(checkpoint["model_state"], strict=False)
model = model.to(device)
model.eval()

with torch.no_grad():
    output = model(image).view(-1)
print(output)
print(output.argmax())
print(labels[output.argmax()])


tensor([-2.8716,  2.5190, -3.5566, -3.5760,  0.7242, -3.0702,  0.2850,  4.0595,
        -5.3640, -2.5182,  1.7910,  0.2871,  3.9205, -3.3784,  0.2304,  5.9378,
         2.1039, -3.1820,  1.7114,  0.0284, -3.0500, -4.7584, -2.4813,  0.1754,
         4.5159,  1.0939,  0.5579,  2.6773, -4.6959,  0.9810, -5.8607,  3.3202,
        11.3435,  8.0537, -6.3752, -2.0821,  0.4500,  3.1894, -0.7948, -4.1438,
        -2.2635, -4.1052, -4.4926,  2.7460,  6.5780, -4.2900, -1.1673,  0.7535,
         0.3452, -2.5385,  3.2370, -3.2789,  3.3606, -3.1844, -6.1738, -1.0989,
        -5.6123,  0.8601, -2.1904, -2.7806, -1.1661, -1.4084, -0.6295, -4.4330,
         0.1510,  1.0303, -5.1813,  2.3988, -4.4285,  4.5388, -0.6637, -2.1544,
         0.1239,  1.1253,  5.0819,  1.4384, -0.2407,  0.6803, -6.0795,  1.8639,
         1.7042, -1.9313,  0.4745, -1.8868,  3.5548,  1.4964, 16.7311,  5.4078,
         3.6670, -2.1254, -6.0045, -6.1703, 11.4196,  4.6421, -3.6557, 26.9912,
         0.5101, -3.5924, -7.4640,  3.99

In [5]:
# export to mobile lite model
model.eval()
from torch.utils.mobile_optimizer import optimize_for_mobile
quant_model = torch.quantization.convert(model)
script_model = torch.jit.script(quant_model)
script_model_optimized = optimize_for_mobile(script_model)
script_model_optimized._save_for_lite_interpreter("../models/final.ptl")